In [12]:
import numpy as np
import os

# 定义文件夹路径和文件名
folder_path = r"C:\Users\wangaochong\Desktop\test5\study_area\area1\depth_test\img_train_out"  # 替换为包含.npy文件的文件夹的路径
file_name = '018199_22.24471041_114.15858540_202303_23_119.npy'  # 替换为要读取的.npy文件的文件名

# 读取.npy文件
file_path = os.path.join(folder_path, file_name)
disparity_map = np.load(file_path)

# 定义要获取像素值的坐标，选了10个点来获取参数
# 11,1908,540,0.09961529076099396,23；
# 14,1226,565,0.09251735359430313,11.8;
# 17,1523,620,0.25859344005584717,6.5;
# 590,1206,570,0.11260422319173813,12.8
# 883,1833,575,0.1417636126279831,11.6
# 1026,904,585,0.11680153012275696,10.5
# 1062,475,705,0.2326141893863678,3.4
# 18966,1829,573,0.14071612060070038,10.7
# 17620,1175,548,0.08917085826396942,13.8
# 18199,1338,580,0.12774212658405304,7.8
pixel_x = 1338  # 替换为你想要获取的像素点的x坐标
pixel_y = 580  # 替换为你想要获取的像素点的y坐标

# 获取像素点的值作为disparity
disparity_value = disparity_map[pixel_y, pixel_x]

print(f"Disparity at pixel ({pixel_x}, {pixel_y}): {disparity_value}")


Disparity at pixel (1338, 580): 0.12774212658405304


如果你有更多的数据点，可以使用线性回归来估计baseline和focal_length的值。以下是一个使用Python的示例函数，可以处理多个数据点：这个函数使用了非线性最小二乘法来拟合深度和视差之间的关系，以估计baseline和focal_length的值。确保替换disparity_values和depth_values中的实际值，并使用正确的单位。这个函数可以适应任意数量的数据点，只要数据点数量大于等于2且具有匹配的深度和视差值。

In [14]:
import numpy as np
from scipy.optimize import curve_fit

def depth_to_disparity(depth, baseline, focal_length):
    return (baseline * focal_length) / depth

def fit_baseline_and_focal_length(disparity_values, depth_values):
    if len(disparity_values) != len(depth_values) or len(disparity_values) < 2:
        raise ValueError("You need at least two data points with matching disparity and depth values to fit baseline and focal length.")
    
    # Convert data to numpy arrays
    disparity_values = np.array(disparity_values)
    depth_values = np.array(depth_values)
    
    # Define the fitting function
    def func(depth, baseline, focal_length):
        return (baseline * focal_length) / depth

    # Fit the function to the data to estimate baseline and focal_length
    popt, _ = curve_fit(func, depth_values, disparity_values)
    
    baseline, focal_length = popt
    return baseline, focal_length

# Example usage with more data points:
disparity_values = [0.09961529076099396, 0.09251735359430313, 0.25859344005584717, 0.11260422319173813,0.1417636126279831,0.11680153012275696,0.2326141893863678
                    ,0.14071612060070038,0.08917085826396942,0.12774212658405304]  # Replace with actual disparity values
depth_values = [23, 11.8, 6.5, 12.8,11.6,10.5,3.4,10.7,13.8,7.8]  # Replace with actual depth values

baseline, focal_length = fit_baseline_and_focal_length(disparity_values, depth_values)
print("Estimated Baseline:", baseline, "Estimated Focal Length:", focal_length)


Estimated Baseline: 1.16418045821547 Estimated Focal Length: 0.9398960573839849


现在已知baseline和focal_length的值，你可以编写一个函数来计算给定像素点的深度距离。使用上面提供的公式 depth = (baseline * focal_length) / disparity_value，以下是一个测试参数效果的函数：

In [17]:
import numpy as np
import os

def calculate_depth_from_disparity(disparity_map, pixel_x, pixel_y, baseline, focal_length):
    # 获取像素点的视差值
    disparity_value = disparity_map[pixel_y, pixel_x]
    
    # 计算深度距离（米）
    depth = (baseline * focal_length) / disparity_value
    
    return depth

# 定义文件夹路径和文件名
folder_path = r'C:\Users\wangaochong\Desktop\test5\study_area\area1\depth_test\img_test_out'  # 替换为包含.npy文件的文件夹的路径
file_name = '000990_22.23881931_114.15207800_202301_7_217.npy'  # 替换为要读取的.npy文件的文件名

# 读取.npy文件
file_path = os.path.join(folder_path, file_name)

disparity_map = np.load(file_path)

# 已知的baseline和focal_length的值
baseline = 1.16418045821547
focal_length = 0.9398960573839849

# 定义要获取像素值的坐标
pixel_x = 1382  # 替换为你想要获取的像素点的x坐标
pixel_y = 595  # 替换为你想要获取的像素点的y坐标

# 使用函数计算深度距离
depth = calculate_depth_from_disparity(disparity_map, pixel_x, pixel_y, baseline, focal_length)
print(f"Depth at pixel ({pixel_x}, {pixel_y}): {depth} meters")


Depth at pixel (1382, 595): 9.675885433383934 meters
